In [ ]:
from tkinter import *
import tkinter as tk
import pandas as pd
from pandastable import Table, TableModel, config
import threading

class App(Frame, threading.Thread):
        def __init__(self, parent=None):
            self.parent = parent
            threading.Thread.__init__(self)
            self.start()
            
        def callback(self):
            self.root.quit()
            
        def run(self):
            Frame.__init__(self)
            self.main = self.master
            self.main.geometry('600x400+200+100')
            self.main.title('Optimizer')
            self.f = Frame(self.main)
            self.f.pack(fill=BOTH,expand=1)
            df = pd.DataFrame()
            self.table = pt = Table(self.f, dataframe=df,
                                    showtoolbar=True, showstatusbar=True)
            pt.show() 
            options = {'colheadercolor':'green','floatprecision': 5} # set some options
            config.apply_options(options, pt)
            pt.show()

            self.mainloop()

In [ ]:
import time

app = App()

def do_something(seconds):                                          # This is a sample function that represents any arbitrary task
    global app
    results = pd.DataFrame({'test': seconds}, index=[0])
    app.table.model.df = pd.concat([app.table.model.df, results], ignore_index=True, axis=0)

    app.table.redraw()
    
    time.sleep(seconds) 
    return 'Done'


time.sleep(1) 

for i in range(10):
    do_something(1)

In [ ]:
import concurrent.futures
import time

app = App()
time.sleep(1)
start = time.perf_counter()                                         # Record when the code started running

def do_something(seconds):                                          # This is a sample function that represents any arbitrary task
    global app
    results = pd.DataFrame({'test': seconds, 'test2': 2}, index=[0])
    app.table.model.df = pd.concat([app.table.model.df, results], ignore_index=True, axis=0)

    app.table.redraw()
    
    time.sleep(seconds) 
    #return 'Done'

# Parallelization block. Here's where the code calls functions and queues them for parallel execution
with concurrent.futures.ThreadPoolExecutor() as executor:          # This syntax helps to automate a few things under the hood
    print("Maximum number of parallel processes: " + str(executor._max_workers))
    results = [executor.submit(do_something, 1) for _ in range(100)] # List comprehension to create a list of executions for them to be used later. This is also the part that queues the functions for parallel execution
    #for f in concurrent.futures.as_completed(results):              # Iteration through list of executions and will only end when all functions end
        #print(f.result())                                           # .results() will return the function output when if finishes executing. In our case, this will be the model's predictions

finish = time.perf_counter()                                        # Records when the code finished running

print(f'Finished in {round(finish-start, 2)} second(s)')            # Find out how long the code ran for